In [1]:
import mysql.connector as msql

sql_password = "Password"


def getUserByWord(word):
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password=sql_password)
    cursor = conn.cursor()
    query = f"SELECT * FROM users_data WHERE name LIKE '%{word}%' or screen_name LIKE '%{word}%';"
    cursor.execute(query)
    a=cursor.fetchall()
    cursor.close()
    conn.close()
    return make_dict_list(a)

def getUserByID(word):
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password=sql_password)
    cursor = conn.cursor()
    query = f"SELECT * FROM users_data WHERE id = '{word}';"
    cursor.execute(query)
    a=cursor.fetchall()
    cursor.close()
    conn.close()
    return make_dict(a[0]) if len(a)>0 else {}

C:\Users\mafaz\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.0) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from pymongo import MongoClient, DESCENDING

drop_field = {'_id': 0}

def getTweetbyID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"id": word}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
        
    return a[0] if len(a)>0 else {}

def getTweetsbyString(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"text": {"$regex": word, "$options": "i"} }
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)  
    return a

def getTweetsbyHashtag(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"hashtags": {"$regex": f"{word}", "$options": "i"}}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i) 
    return a

def getTweetsbyUserIDsList(lst):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"user.id": {"$in": lst}}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
    return a

def getTweetsbyUserID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"user.id": word}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
    return a

def getRetweetsbyUserID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Retweets"]

    query = {"user.id": word}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
    return a

def getRetweetsByOriginalTweetID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Retweets"]

    query = {"retweeted_status.id": word}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
    return a

In [4]:
def time_filter(data, stime, etime, fetch_time):
    if stime=="na" and etime=="na":
        return {'data': data, 'fetch_time': fetch_time }
    
    if stime=="na":
        res = []
        etime = int(etime)/1000
        for tweet in data:
            if tweet['created_at'] <= etime:
                res.append(tweet)
        return {'data': res, 'fetch_time': fetch_time }

    if etime=="na":
        res = []
        stime = int(stime)/1000
        for tweet in data:
            if tweet['created_at'] >= stime:
                res.append(tweet)
        return {'data': res, 'fetch_time': fetch_time }
    
    res = []
    stime, etime = int(stime)/1000, int(etime)/1000
    for tweet in data:
        if stime <= tweet['created_at'] <= etime:
            res.append(tweet)
    return {'data': res, 'fetch_time': fetch_time }

In [3]:
col_keys = [ 'id', 'name', 'screen_name', 'location', 'url', 'description', 'verified','followers_count',
            'friends_count', 'listed_count', 'favourites_count', 'statuses_count', 'created_at' ]

def make_dict(item):
    kv = {}
    for i in range(len(col_keys)):
        kv[col_keys[i]] = item[i]
    return kv
    
def make_dict_list(lst):
    res = []
    for item in lst:
        res.append(make_dict(item))  
    return res

def extract_val(key, lst):
    res = []
    for item in lst:
        res.append(item[key])
    return res